<a href="https://colab.research.google.com/github/beratcmn/tensorflow_quickdraw_recognition/blob/main/doodle_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing libraries

In [1]:
!pip install quickdraw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

from pathlib import Path
from quickdraw import QuickDrawDataGroup

tf.__version__

'2.11.0'

### Getting the doodles

In [3]:
image_size = (64, 64)
categories = ["airplane", "apple", "bicycle", "car", "cat", "cloud", "dog", "hamburger", "fish", "flower", "banana", "bird", "eye", "fork", "hat"]
max_drawings = 5000

def generate_class_images(name, max_drawings, recognized):
    directory = Path("data/" + name)

    if not directory.exists():
        directory.mkdir(parents=True)

    images = QuickDrawDataGroup(name, max_drawings=max_drawings, recognized=recognized)
    for img in images.drawings:
        filename = directory.as_posix() + "/" + str(img.key_id) + ".png"
        img.get_image(stroke_width=3).resize(image_size).save(filename)

for label in categories:
    generate_class_images(label, max_drawings=max_drawings, recognized=True)

downloading airplane from https://storage.googleapis.com/quickdraw_dataset/full/binary/airplane.bin
download complete
loading airplane drawings
load complete
downloading apple from https://storage.googleapis.com/quickdraw_dataset/full/binary/apple.bin
download complete
loading apple drawings
load complete
downloading bicycle from https://storage.googleapis.com/quickdraw_dataset/full/binary/bicycle.bin
download complete
loading bicycle drawings
load complete
downloading car from https://storage.googleapis.com/quickdraw_dataset/full/binary/car.bin
download complete
loading car drawings
load complete
downloading cat from https://storage.googleapis.com/quickdraw_dataset/full/binary/cat.bin
download complete
loading cat drawings
load complete
downloading cloud from https://storage.googleapis.com/quickdraw_dataset/full/binary/cloud.bin
download complete
loading cloud drawings
load complete
downloading dog from https://storage.googleapis.com/quickdraw_dataset/full/binary/dog.bin
download comp

### Creating the datasets

In [5]:
batch_size = 32

train_dataset = image_dataset_from_directory(
    "data",
    validation_split=0.2,
    subset="training",
    seed=123,
    color_mode="grayscale",
    image_size=image_size,
    batch_size=batch_size
)

validation_dataset = image_dataset_from_directory(
    "data",
    validation_split=0.2,
    subset="validation",
    seed=123,
    color_mode="grayscale",
    image_size=image_size,
    batch_size=batch_size
)

Found 75000 files belonging to 15 classes.
Using 60000 files for training.
Found 75000 files belonging to 15 classes.
Using 15000 files for validation.
